# Setup and Data Cleaning

This notebook loads the Swiggy reviews CSV, normalizes the data, and saves it to Parquet format for efficient processing.


In [1]:
import polars as pl
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone, timedelta
import json

# Set up paths
DATA_DIR = Path("../data")
CSV_FILE = Path("../swiggy_scraped.csv")
OUTPUT_FILE = DATA_DIR / "reviews_clean.parquet"

IST_TZ = timezone(timedelta(hours=5, minutes=30))
START_DATE = datetime(2024, 6, 1, tzinfo=IST_TZ)

DATA_DIR.mkdir(exist_ok=True)

print("✓ Setup complete")


✓ Setup complete


## Load CSV Data


In [2]:
print("Loading CSV file...")
df = pl.read_csv(CSV_FILE)

print(f"✓ Loaded {len(df):,} rows")
print(f"Columns: {df.columns}")
print("\nFirst few rows:")
print(df.head(3))


Loading CSV file...
✓ Loaded 264,800 rows
Columns: ['reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion']

First few rows:
shape: (3, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ reviewId  ┆ userName  ┆ userImage ┆ content   ┆ … ┆ at        ┆ replyCont ┆ repliedAt ┆ appVersi │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ent       ┆ ---       ┆ on       │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆ str       ┆ ---       ┆ str       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ cf39a850- ┆ Subhash   ┆ https://p ┆ good 😊   ┆ … ┆ 2025-10-2 ┆ Hey,      ┆ 2025-10-2 ┆ 4.94.0   │
│ 943d-47b1 ┆ Mandal    ┆ lay-lh.go ┆           ┆   ┆ 6        

## Data Cleaning

Parse datetimes, normalize text, and create computed columns


In [3]:
# Parse datetime with IST timezone
# IST is UTC+5:30
ist_offset = timedelta(hours=5, minutes=30)

df = df.with_columns([
    # Parse created_at (AT column)
    pl.col("at").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False)
        .dt.replace_time_zone("Asia/Kolkata")
        .alias("created_at"),
    
    # Parse replied_at
    pl.when(pl.col("repliedAt") != "").then(
        pl.col("repliedAt").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False)
            .dt.replace_time_zone("Asia/Kolkata")
    ).otherwise(None).alias("replied_at"),
    
    # Rename key columns
    pl.col("reviewId").alias("review_id"),
    pl.col("userName").alias("user_name"),
    pl.col("content").alias("content_raw"),
    
    # Create clean version
    pl.col("content")
        .str.strip_chars()
        .str.to_lowercase()
        .alias("content_clean"),
    
    # Cast numeric types
    pl.col("score").cast(pl.Int8),
    pl.col("thumbsUpCount").cast(pl.Int32).alias("thumbs_up"),
    
    # Handle app_version (may be empty)
    pl.when(pl.col("reviewCreatedVersion") == "").then(None)
        .otherwise(pl.col("reviewCreatedVersion"))
        .alias("app_version"),
    
    pl.col("replyContent").alias("reply_text"),
])

print("✓ Datetime and naming transformations complete")


✓ Datetime and naming transformations complete


In [4]:
# Add computed columns
df = df.with_columns([
    # Length in tokens (approx word count)
    pl.col("content_clean")
        .str.split(" ")
        .list.len()
        .alias("length_tokens"),
    
    # Flag for short reviews (< 10 words)
    (pl.col("content_clean").str.split(" ").list.len() < 10)
        .alias("is_short"),
    
    # Extract date for partitioning
    pl.col("created_at").dt.date().alias("dt"),
])

print("✓ Computed columns added")


✓ Computed columns added


## Select Final Columns


In [7]:
# Select final schema
pre_filter_rows = len(df)

df = df.filter(
    pl.col("created_at") >= pl.lit(START_DATE).dt.convert_time_zone("Asia/Kolkata")
)

# df = df.filter(pl.col('created_at') >= START_DATE)
post_filter_rows = len(df)
if post_filter_rows == 0:
    raise ValueError("No reviews remain after applying June 2024+ filter. Check input data.")
print(f"✓ Filtered to June 2024 onward: {post_filter_rows:,} rows (removed {pre_filter_rows - post_filter_rows:,})")

df_final = df.select([
    "review_id",
    "user_name",
    "content_raw",
    "content_clean",
    "score",
    "thumbs_up",
    "created_at",
    "reply_text",
    "replied_at",
    "app_version",
    "length_tokens",
    "is_short",
    "dt"
])

print(f"✓ Final dataset: {len(df_final):,} rows, {len(df_final.columns)} columns")
print(f"Schema: {df_final.schema}")


✓ Filtered to June 2024 onward: 225,918 rows (removed 38,882)
✓ Final dataset: 225,918 rows, 13 columns
Schema: Schema([('review_id', String), ('user_name', String), ('content_raw', String), ('content_clean', String), ('score', Int8), ('thumbs_up', Int32), ('created_at', Datetime(time_unit='us', time_zone='Asia/Kolkata')), ('reply_text', String), ('replied_at', Datetime(time_unit='us', time_zone='Asia/Kolkata')), ('app_version', String), ('length_tokens', UInt32), ('is_short', Boolean), ('dt', Date)])


## Data Statistics


In [8]:
print("📊 Data Statistics")
print("=" * 60)

# Date range
print(f"\n📅 Date Range:")
min_date = df_final['created_at'].min()
print(f"  Min (expected ≥ {START_DATE.date()}): {min_date}")
if min_date < START_DATE:
    print("⚠️ Warning: Minimum date is earlier than June 1 2024. Check filtering logic.")
max_date = df_final['created_at'].max()
print(f"  Max: {max_date}")
print(f"  Unique dates: {df_final['dt'].unique().len()}")

# Rating distribution
print(f"\n⭐ Score Distribution:")
print(df_final.group_by("score").agg(pl.count()).sort("score"))

# Length statistics
print(f"\n📝 Review Length:")
print(f"  Average tokens: {df_final['length_tokens'].mean():.1f}")
print(f"  Short reviews (<10 words): {df_final['is_short'].sum():,} ({df_final['is_short'].mean()*100:.1f}%)")

# Sample reviews
print(f"\n📄 Sample Reviews:")
for i, row in enumerate(df_final.head(5).iter_rows(named=True)):
    print(f"\n{i+1}. Score: {row['score']}, Length: {row['length_tokens']} tokens")
    print(f"   {row['content_raw'][:100]}...")


📊 Data Statistics

📅 Date Range:
  Min (expected ≥ 2024-06-01): 2024-06-01 00:15:24+05:30
  Max: 2025-10-26 13:26:07+05:30
  Unique dates: 513

⭐ Score Distribution:
shape: (5, 2)
┌───────┬────────┐
│ score ┆ count  │
│ ---   ┆ ---    │
│ i8    ┆ u32    │
╞═══════╪════════╡
│ 1     ┆ 81203  │
│ 2     ┆ 6310   │
│ 3     ┆ 8831   │
│ 4     ┆ 18692  │
│ 5     ┆ 110882 │
└───────┴────────┘

📝 Review Length:
  Average tokens: 11.6
  Short reviews (<10 words): 165,229 (73.1%)

📄 Sample Reviews:

1. Score: 3, Length: 2 tokens
   good 😊...

2. Score: 5, Length: 2 tokens
   Great apps...

3. Score: 5, Length: 1 tokens
   Good...

4. Score: 5, Length: 1 tokens
   Good...

5. Score: 4, Length: 2 tokens
   it's good...


/tmp/ipykernel_15428/1453187835.py:16: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  print(df_final.group_by("score").agg(pl.count()).sort("score"))


## Save to Parquet


In [9]:
# Remove dt column before saving (it's for partitioning only)
df_to_save = df_final.drop("dt")

# Save to Parquet
df_to_save.write_parquet(OUTPUT_FILE)

print(f"✓ Saved {len(df_to_save):,} rows to {OUTPUT_FILE}")
print(f"  File size: {OUTPUT_FILE.stat().st_size / (1024**2):.2f} MB")


✓ Saved 225,918 rows to ../data/reviews_clean.parquet
  File size: 18.95 MB


## Verification

Verify the saved Parquet file


In [10]:
# Verify saved file
df_loaded = pl.read_parquet(OUTPUT_FILE)

print(f"✓ Loaded {len(df_loaded):,} rows from Parquet")
print(f"Columns: {df_loaded.columns}")
print("\nSample:")
print(df_loaded.head(2))


✓ Loaded 225,918 rows from Parquet
Columns: ['review_id', 'user_name', 'content_raw', 'content_clean', 'score', 'thumbs_up', 'created_at', 'reply_text', 'replied_at', 'app_version', 'length_tokens', 'is_short']

Sample:
shape: (2, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ review_id ┆ user_name ┆ content_r ┆ content_c ┆ … ┆ replied_a ┆ app_versi ┆ length_to ┆ is_short │
│ ---       ┆ ---       ┆ aw        ┆ lean      ┆   ┆ t         ┆ on        ┆ kens      ┆ ---      │
│ str       ┆ str       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ bool     │
│           ┆           ┆ str       ┆ str       ┆   ┆ datetime[ ┆ str       ┆ u32       ┆          │
│           ┆           ┆           ┆           ┆   ┆ μs, Asia/ ┆           ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ Kolkata]  ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═════

In [11]:
df_loaded.head()

review_id,user_name,content_raw,content_clean,score,thumbs_up,created_at,reply_text,replied_at,app_version,length_tokens,is_short
str,str,str,str,i8,i32,"datetime[μs, Asia/Kolkata]",str,"datetime[μs, Asia/Kolkata]",str,u32,bool
"""cf39a850-943d-47b1-a914-af344f…","""Subhash Mandal""","""good 😊""","""good 😊""",3,0,2025-10-26 13:26:07 IST,"""Hey, thank you for the ratings…",2025-10-26 13:32:41 IST,"""4.94.0""",2,true
"""b5144829-dcbc-41aa-bb47-107488…","""Macdonald Thyrniang""","""Great apps""","""great apps""",5,0,2025-10-26 13:24:48 IST,"""Hey there! It's a bliss to see…",2025-10-26 13:32:41 IST,"""4.94.0""",2,true
"""6cf287b1-a486-43c4-bff9-76f6e8…","""Hemangi Ahir""","""Good""","""good""",5,0,2025-10-26 13:24:18 IST,"""Hey, thanks for the rating! Th…",2025-10-26 13:32:42 IST,"""4.94.0""",1,true
"""99fc5c31-b1b6-44ad-a956-3dffbf…","""JOHN WILLIAM""","""Good""","""good""",5,0,2025-10-26 13:20:51 IST,"""Hey, we are glad to have patro…",2025-10-26 13:22:33 IST,"""4.94.0""",1,true
"""5e7d0b8b-0383-400e-a9c6-2140e2…","""Siddiqui Siddiqui""","""it's good""","""it's good""",4,0,2025-10-26 13:20:30 IST,"""Hello, thank you so much for t…",2025-10-26 13:22:33 IST,"""4.93.4""",2,true
